In [8]:
from preprocessing.preprocessing_label_encoding import  get_preprocessed_brfss_dataset_label_encoded_train_test_split, get_preprocessed_brfss_dataset_label_encoded_train_test_split_undersampled
from preprocessing.preprocessing import diabetes_columns
from sklearn.tree import DecisionTreeClassifier
import graphviz
from sklearn import tree
from IPython.core.display_functions import display
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, cross_val_predict
from sklearn.metrics import classification_report
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
data_train, data_test, target_train, target_test = get_preprocessed_brfss_dataset_label_encoded_train_test_split()
cross_validation = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
data_train.head(10)

,GenHealth,PhysHealth,MentHealth,Healthcare,MedCost,Checkup,HighBP,HighChol,HeartAttack,AngiCoro,...,Height,Weight,BMI,Education,Alcohol,Smoking,FruitCons,VegetCons,PhysActivity,Muscles
32353,1.0,0.0,0.100000,1.0,2.0,1.0,3.0,2.0,2.0,2.0,...,0.733624,0.222218,2.0,4.0,1.0,4.0,1.0,1.0,3.0,1.0
192208,2.0,0.0,0.000000,1.0,2.0,2.0,3.0,2.0,2.0,2.0,...,0.777293,0.250371,2.0,4.0,1.0,2.0,1.0,1.0,3.0,1.0
171948,3.0,0.0,1.000000,1.0,2.0,1.0,1.0,1.0,2.0,2.0,...,0.000000,0.000000,0.0,4.0,1.0,4.0,2.0,1.0,3.0,2.0
68153,2.0,0.0,0.000000,1.0,2.0,1.0,3.0,0.0,2.0,2.0,...,0.820961,0.336450,3.0,4.0,1.0,4.0,2.0,1.0,1.0,2.0
122769,4.0,0.5,0.666667,1.0,2.0,4.0,1.0,1.0,2.0,2.0,...,0.720524,0.341142,4.0,2.0,1.0,1.0,2.0,2.0,4.0,2.0
178211,1.0,0.0,0.000000,1.0,2.0,4.0,3.0,0.0,2.0,2.0,...,0.786026,0.289495,3.0,2.0,2.0,3.0,1.0,1.0,3.0,1.0
200439,3.0,0.0,0.000000,1.0,2.0,1.0,3.0,2.0,2.0,2.0,...,0.742358,0.344282,4.0,3.0,1.0,4.0,2.0,2.0,3.0,2.0
75248,2.0,0.0,0.000000,1.0,2.0,1.0,1.0,2.0,2.0,2.0,...,0.698690,0.226910,3.0,4.0,1.0,4.0,2.0,2.0,3.0,2.0
176426,1.0,0.0,0.000000,1.0,2.0,3.0,3.0,2.0,2.0,2.0,...,0.799127,0.430361,4.0,2.0,1.0,4.0,2.0,2.0,2.0,1.0
38656,5.0,1.0,0.500000,1.0,2.0,2.0,1.0,1.0,1.0,1.0,...,0.755459,0.297326,3.0,4.0,1.0,3.0,1.0,1.0,1.0,2.0


In [4]:
decision_tree_classifier = DecisionTreeClassifier()
decision_tree_classifier.fit(data_train, target_train)

accuracy = cross_val_score(decision_tree_classifier, data_train, target_train, cv=cross_validation, scoring='accuracy')

for i, acc in enumerate(accuracy):
    print("Fold {}: Accuracy = {}%".format(i, acc * 100.0))

print("Average Accuracy = {}%".format(accuracy.mean() * 100.0))

Fold 0: Accuracy = 77.1591843802762%
Fold 1: Accuracy = 77.10290488765747%
Fold 2: Accuracy = 76.68730248062687%
Fold 3: Accuracy = 76.86479934196285%
Fold 4: Accuracy = 76.9513831767609%
Fold 5: Accuracy = 77.30637689943288%
Fold 6: Accuracy = 77.04229620329885%
Fold 7: Accuracy = 77.39296073423093%
Fold 8: Accuracy = 76.84648021473721%
Fold 9: Accuracy = 77.08892544809073%
Average Accuracy = 77.0442613767075%


In [ ]:
dot_data = tree.export_graphviz(decision_tree_classifier, out_file=None,
                                feature_names=data_train.columns,
                                class_names=diabetes_columns,
                                filled=True, rounded=True, special_characters=True)
graph = graphviz.Source(dot_data)

In [ ]:
display(graph)

In [5]:
decision_tree_classifier = DecisionTreeClassifier()
parameters = {
    'max_depth': range(5, 28),
    'splitter': ["best", "random"],
    'criterion': ["gini", "entropy", "log_loss"]
}

grid_search_estimator = GridSearchCV(decision_tree_classifier, parameters, scoring='accuracy', cv=cross_validation, return_train_score=False)
grid_search_estimator.fit(data_train, target_train)

results = pd.DataFrame(grid_search_estimator.cv_results_)
display(results)

print(f"best score is {grid_search_estimator.best_score_} with params {grid_search_estimator.best_params_}")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_splitter,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.296333,0.008722,0.003868,0.001020,gini,5,best,"{'criterion': 'gini', 'max_depth': 5, 'splitte...",0.853933,0.851898,...,0.854842,0.854366,0.853587,0.853543,0.854582,0.854879,0.853927,0.853884,0.000845,10
1,0.165737,0.009328,0.003516,0.000479,gini,5,random,"{'criterion': 'gini', 'max_depth': 5, 'splitte...",0.852158,0.851162,...,0.855924,0.852504,0.852245,0.853457,0.850167,0.854879,0.850117,0.852590,0.001787,29
2,0.347150,0.009114,0.003568,0.000429,gini,6,best,"{'criterion': 'gini', 'max_depth': 6, 'splitte...",0.855362,0.852201,...,0.855795,0.854409,0.853846,0.854626,0.855059,0.856048,0.854923,0.854594,0.001079,1
3,0.198906,0.009556,0.003433,0.000273,gini,6,random,"{'criterion': 'gini', 'max_depth': 6, 'splitte...",0.855275,0.851292,...,0.853803,0.854323,0.853587,0.853543,0.854496,0.853147,0.852888,0.853620,0.001013,22
4,0.414432,0.010676,0.003675,0.000319,gini,7,best,"{'criterion': 'gini', 'max_depth': 7, 'splitte...",0.855102,0.851812,...,0.856184,0.854842,0.853717,0.854539,0.854669,0.855745,0.854403,0.854482,0.001150,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,0.501391,0.009534,0.006895,0.000382,log_loss,25,random,"{'criterion': 'log_loss', 'max_depth': 25, 'sp...",0.783540,0.781029,...,0.781636,0.783627,0.783411,0.778822,0.781679,0.784267,0.785566,0.782569,0.001814,120
134,0.891026,0.002497,0.006614,0.000242,log_loss,26,best,"{'criterion': 'log_loss', 'max_depth': 26, 'sp...",0.780553,0.775142,...,0.774189,0.777177,0.779687,0.776917,0.779428,0.774396,0.778292,0.777235,0.002126,133
135,0.511144,0.005260,0.006923,0.000194,log_loss,26,random,"{'criterion': 'log_loss', 'max_depth': 26, 'sp...",0.783627,0.774059,...,0.784839,0.774016,0.784623,0.777177,0.782545,0.779202,0.782752,0.780101,0.003936,125
136,0.892878,0.002251,0.006583,0.000181,log_loss,27,best,"{'criterion': 'log_loss', 'max_depth': 27, 'sp...",0.777653,0.776570,...,0.774189,0.777003,0.780034,0.775964,0.779168,0.772837,0.776994,0.776452,0.002149,136


best score is 0.8545941857684177 with params {'criterion': 'gini', 'max_depth': 6, 'splitter': 'best'}


### Let's see how good the best classifier really is

In [6]:
decision_tree_classifier_best = DecisionTreeClassifier(max_depth=7, splitter="best", criterion="gini")
prediction = cross_val_predict(decision_tree_classifier_best, data_train, target_train, cv=cross_validation)
report = classification_report(target_train, prediction)
print(report)

              precision    recall  f1-score   support

         1.0       0.54      0.15      0.23     28796
         2.0       0.00      0.00      0.00      1965
         3.0       0.87      0.98      0.92    196381
         4.0       0.50      0.00      0.00      3846

    accuracy                           0.85    230988
   macro avg       0.48      0.28      0.29    230988
weighted avg       0.81      0.85      0.81    230988



### Test with undersampling

In [9]:
data_train, data_test, target_train, target_test = get_preprocessed_brfss_dataset_label_encoded_train_test_split_undersampled()

decision_tree_classifier = DecisionTreeClassifier()
parameters = {
    'max_depth': range(5, 28),
    'splitter': ["best", "random"],
    'criterion': ["gini", "entropy", "log_loss"]
}

grid_search_estimator = GridSearchCV(decision_tree_classifier, parameters, scoring='accuracy', cv=cross_validation, return_train_score=False)
grid_search_estimator.fit(data_train, target_train)

results = pd.DataFrame(grid_search_estimator.cv_results_)
display(results)

print(f"best score is {grid_search_estimator.best_score_} with params {grid_search_estimator.best_params_}")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_splitter,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010755,0.000540,0.000687,0.000168,gini,5,best,"{'criterion': 'gini', 'max_depth': 5, 'splitte...",0.493639,0.466921,...,0.473282,0.465649,0.482188,0.451654,0.494911,0.438931,0.452926,0.465522,0.019942,12
1,0.005526,0.000446,0.000551,0.000064,gini,5,random,"{'criterion': 'gini', 'max_depth': 5, 'splitte...",0.483461,0.435115,...,0.479644,0.454198,0.478372,0.460560,0.460560,0.447837,0.472010,0.458015,0.022007,25
2,0.012257,0.000328,0.000641,0.000127,gini,6,best,"{'criterion': 'gini', 'max_depth': 6, 'splitte...",0.484733,0.460560,...,0.455471,0.430025,0.468193,0.449109,0.484733,0.430025,0.456743,0.456107,0.018518,28
3,0.006377,0.000384,0.000622,0.000142,gini,6,random,"{'criterion': 'gini', 'max_depth': 6, 'splitte...",0.487277,0.450382,...,0.493639,0.455471,0.470738,0.473282,0.466921,0.459288,0.479644,0.467303,0.016550,9
4,0.014022,0.000626,0.000647,0.000094,gini,7,best,"{'criterion': 'gini', 'max_depth': 7, 'splitte...",0.487277,0.459288,...,0.468193,0.458015,0.484733,0.452926,0.478372,0.441476,0.452926,0.462977,0.015166,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,0.018117,0.000270,0.000695,0.000123,log_loss,25,random,"{'criterion': 'log_loss', 'max_depth': 25, 'sp...",0.449109,0.389313,...,0.396947,0.353690,0.417303,0.366412,0.394402,0.394402,0.403308,0.392748,0.026560,134
134,0.032580,0.000483,0.000692,0.000064,log_loss,26,best,"{'criterion': 'log_loss', 'max_depth': 26, 'sp...",0.435115,0.422392,...,0.414758,0.395674,0.408397,0.379135,0.399491,0.388041,0.372774,0.401654,0.018261,106
135,0.018258,0.000460,0.000694,0.000138,log_loss,26,random,"{'criterion': 'log_loss', 'max_depth': 26, 'sp...",0.404580,0.398219,...,0.445293,0.368957,0.393130,0.370229,0.390585,0.377863,0.371501,0.392875,0.022196,133
136,0.032443,0.000577,0.000867,0.000465,log_loss,27,best,"{'criterion': 'log_loss', 'max_depth': 27, 'sp...",0.427481,0.410941,...,0.419847,0.400763,0.394402,0.376590,0.400763,0.399491,0.372774,0.399109,0.016511,118


best score is 0.46921119592875316 with params {'criterion': 'gini', 'max_depth': 8, 'splitter': 'best'}
